In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [8]:
df= pd.read_csv("C:\\Users\\KUNAL\\.cache\\kagglehub\\datasets\\saurabhbadole\\bank-customer-churn-prediction-dataset\\versions\\2\\Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
gen_encoder = LabelEncoder()
df["Gender"] = gen_encoder.fit_transform(df["Gender"])


In [13]:
df.Gender.value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [31]:
geo_encoder = OneHotEncoder(sparse_output=False)
geo_encoded_col = geo_encoder.fit_transform(df[["Geography"]])

In [32]:
geo_df = pd.DataFrame(geo_encoded_col, columns=geo_encoder.get_feature_names_out(["Geography"]))

In [34]:
df = pd.concat([df, geo_df], axis=1)
df = df.drop(columns=["Geography"], axis=1)

In [35]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [36]:
X = df.drop("EstimatedSalary", axis=1)
y = df["EstimatedSalary"]

In [37]:
X_train, X_test, y_train , y_test = train_test_split(X, y,  test_size=0.2, random_state=42)

In [38]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [39]:
with open("Gen.pkl", "wb") as file:
    pickle.dump(gen_encoder, file)

with open("Geo.pkl", "wb") as file:
    pickle.dump(geo_encoder, file)

with open("Scaler.pkl", "wb") as file:
    pickle.dump(sc, file)

# ANN for Regression

In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [43]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation="linear")
]
)

In [46]:
model.compile(optimizer="adam", loss="mean_absolute_error", metrics=["mae"])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "logs/reg/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)

early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

In [50]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorboard_callback, early_stopping],
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 99483.6172 - mae: 99483.6172 - val_loss: 98485.1875 - val_mae: 98485.1875
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99343.8594 - mae: 99343.8594 - val_loss: 96743.3750 - val_mae: 96743.3750
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 97077.4453 - mae: 97077.4453 - val_loss: 92313.7422 - val_mae: 92313.7422
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 91292.2734 - mae: 91292.2734 - val_loss: 85055.3281 - val_mae: 85055.3281
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 86514.2734 - mae: 86514.2734 - val_loss: 76068.6484 - val_mae: 76068.6484
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 74586.8047 - mae: 74586.8047 - val_loss: 67089.9844 - val_mae: 67089.9844
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 66304.6875 - mae: 66304.6875 - val_loss: 59676.8555 - val_mae: 59676.8555
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - 

In [51]:
%load_ext tensorboard

In [57]:
%tensorboard --logdir logs/reg/20250715-185956

Reusing TensorBoard on port 6008 (pid 27836), started 0:00:05 ago. (Use '!kill 27836' to kill it.)

In [58]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51079.2695 - mae: 51079.2695
Test Loss: 50303.8984375, Test MAE: 50303.8984375


In [59]:
model.save("salary_reg_model.h5")